In [1]:
import sys
from itertools import compress
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.metrics import make_scorer
from audioop import cross
from tabnanny import verbose
from sklearn.model_selection import cross_validate
sys.path.append('..')
from import_library import *
from model_stack import model_list

In [2]:
print("CPU count: {}".format(multiprocessing.cpu_count()))
data_type ='mix' #mean, var, mix
data_value_type = "FC" #ALFF, FC
grid = False
kfold = KFold(n_splits=4)
cross_v = True

CPU count: 8


In [3]:

# 這邊使用到的資料集都是從R篩選後的csv
feature_selection_df_savepath = '../../R/feature_selection_result/'
# dataloader
clinical_data = pd.read_csv('../../R/dataset/clinical_data_2.csv')[['subj', 'AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]']]
raw_data_path = '../../R/dataset/'
next_path = [i for i in os.listdir('../../R/dataset/') if 'MOCA'in i or 'AD8' in i]

# feature selection process
# rank_threshold = 'R-Boruta-Results'
rank_threshold = 80
def feature_filter(patient_value, load_path = None, outcome_name = None, data_type_ = 'mean', data_value_type = None):
    feature_rank_df = pd.read_csv(load_path, index_col = False).rename(columns={'Unnamed: 0': 'feature_name'})
    
    feature_rank_df = feature_rank_df[['feature_name', 'decision']]
    if rank_threshold == 'R-Boruta-Results':
        feature_rank_df = feature_rank_df[feature_rank_df['decision']!='Rejected']
    else:
        feature_rank_df = feature_rank_df.head(rank_threshold)
    patient_value = patient_value[feature_rank_df['feature_name'].tolist()]
    return feature_rank_df['feature_name'].tolist(), patient_value

def loader_(data_path = None ,mat_dtype='FCM_mean', outcome_name = None):
    outcome_name_type = ("AD8" if "AD8" in outcome_name else "MOCA")
    if data_type =='mix':
        if data_value_type =='FC': 
            
            # MOCA客觀認知測驗分數[<=23,>=24]_mean_10000_FC.csv
            pd_tb_1 = pd.read_csv(os.path.join(data_path[0], '{}_mean_{}_{}.csv'.format(outcome_name, 10000, data_value_type))) 
            pd_tb_2 = pd.read_csv(os.path.join(data_path[0], '{}_var_{}_{}.csv'.format(outcome_name, 10000, data_value_type)))
        else:
            
            pd_tb_1, pd_tb_2 = pd.read_csv(os.path.join(data_path[0], 'mean_mdALFF.csv')), pd.read_csv(os.path.join(data_path[0], 'var_mdALFF.csv'))
        pd_patient_index_1, pd_patient_index_2 = pd_tb_1[pd_tb_1.columns[0]]+'_m', pd_tb_2[pd_tb_2.columns[0]]+'_v'
        pd_patient_value_1, pd_patient_value_2 = pd_tb_1[pd_tb_1.columns[1::]], pd_tb_2[pd_tb_2.columns[1::]]
        fd_csv_name = ("R-boruta-{}-{}_{}_feature-selection_result.csv".format(data_value_type, outcome_name_type, 'mean'))
        col_1, pd_patient_value_1 = feature_filter(pd_patient_value_1, load_path = os.path.join(feature_selection_df_savepath, fd_csv_name), outcome_name = outcome_name, data_type_ = 'mean', )
        fd_csv_name = ("R-boruta-{}-{}_{}_feature-selection_result.csv".format(data_value_type, outcome_name_type, 'var'))
        col_2, pd_patient_value_2 = feature_filter(pd_patient_value_2, load_path = os.path.join(feature_selection_df_savepath, fd_csv_name), outcome_name = outcome_name, data_type_ = 'var', )
        print(pd_patient_value_1.shape, pd_patient_value_2.shape)
        pd_patient_value_mix = pd.DataFrame(np.concatenate((pd_patient_value_1, pd_patient_value_2), axis=1), columns=col_1+col_2)
        return col_1+col_2, pd_patient_value_mix.values

    else:
        if data_value_type =='FC': 
            # AD8主觀認知障礙[0-1,>=2]_var_10000_FC.csv
            overall_feature_df = '{}_{}_{}_{}.csv'.format(outcome_name, data_type, 10000, data_value_type)
        else:
            overall_feature_df = '{}_mdALFF.csv'.format(data_type)
        pd_tb = pd.read_csv(os.path.join(data_path[0], overall_feature_df))
        pd_patient_index = pd_tb[pd_tb.columns[0]]
        pd_patient_value = pd_tb[pd_tb.columns[1::]]
        fd_csv_name = ("R-boruta-{}-{}_{}_feature-selection_result.csv".format(data_value_type, outcome_name_type, data_type))
        pd_patient_index, pd_patient_value = feature_filter(pd_patient_value, load_path=os.path.join(feature_selection_df_savepath, fd_csv_name), outcome_name = outcome_name_type)

        return pd_patient_index, pd_patient_value.values

In [4]:
result_dict={'outcome':[], 'model':[], 'Reg:mae':[], 'Reg:mse':[],  'Reg:mape':[] }
grid_stack, scores_stack = [], []
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
if True:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        fxn()
        # classification : AD8 [0-1, >=2]{0,1}, MOCA [<=23, >=24]{0,1}
        # outcome_name = 'MOCA客觀認知測驗分數[<=23,>=24]' #AD8主觀認知障礙[0-1,>=2], MOCA客觀認知測驗分數[<=23,>=24]
        model_stack = model_list(grid=grid)
        
        
        for outcome_name in ["AD8主觀認知障礙[0-1,>=2]", "MOCA客觀認知測驗分數[<=23,>=24]"]:
            for model in model_stack:
                if model in ['MLPR', 'XBGR', 'LGBMR']:
                    result_dict['outcome'].append(outcome_name)
                    result_dict['model'].append(model)
                    label_ = clinical_data[outcome_name]
                    if 'AD8' in outcome_name:
                        label_ = label_+1
                    _, patient_value = loader_(data_path = [raw_data_path, next_path], outcome_name = outcome_name)
                    print("Overall: ",patient_value.shape)
                    print("Feature Number: {}".format(patient_value.shape[1]))
                    if grid==False:
                        print("ALFF Outcome: {}[{}] - cut rank point: {} - GridSearch: {}".format(outcome_name, data_type, rank_threshold, grid))
                        # print("--Start Training {} model--".format(model))
                        if cross_v==True:
                            scoring = {'mae': make_scorer(mean_absolute_error), 'mse': make_scorer(mean_squared_error), 'mape':make_scorer(mean_absolute_percentage_error)}
                            scores = cross_validate(model_stack[model], patient_value, np.array(label_), scoring=scoring, cv = 5,  return_train_score=True)
                            scores_stack.append(scores)
                        else:
                            model = model_stack[model].fit(patient_value, np.array(label_))
                            overall_pred =  model.predict(patient_value)
                        if cross_v==True:
                            result_dict['Reg:mae'].append("{} - {}".format(round(np.min(scores['test_mae']),5), round(np.max(scores['test_mae']),5)))
                            result_dict['Reg:mse'].append("{} - {}".format(round(np.min(scores['test_mse']),5), round(np.max(scores['test_mse']),5)))
                            result_dict['Reg:mape'].append("{} - {}".format(round(np.min(scores['test_mape']),5), round(np.max(scores['test_mape']),5))) 
                        else:
                            result_dict['Reg:mae'].append(round(mean_absolute_error(np.array(label_),overall_pred), 5))
                            result_dict['Reg:mse'].append(round(mean_squared_error(np.array(label_),overall_pred),5))
                            result_dict['Reg:mape'].append(round(mean_absolute_percentage_error(np.array(label_),overall_pred),5))
                    else:
                        print("ALFF Outcome: {}[{}] - cut rank point: {} - GridSearch: {}".format(outcome_name, data_type, rank_threshold, grid))
                        print("--Start Training {} model--".format(model))
                        scoring = {'mae': make_scorer(mean_absolute_error), 'mse': make_scorer(mean_squared_error), 'mape':make_scorer(mean_absolute_percentage_error)}
                        GridModel = GridSearchCV(model_stack[model][0],model_stack[model][1], cv=kfold,  n_jobs = 20, refit = 'neg_mean_absolute_percentage_error', scoring="neg_mean_absolute_percentage_error",
                                                    return_train_score=True)
                        print(patient_value.shape, np.array(label_).shape)
                        GridModel.fit(patient_value, np.array(label_))
                        print('Best parameter : [{}]\n'.format(GridModel.best_params_))
                        # change model parameter
                        param_ = GridModel.best_params_
                        model_stack[model][0].set_params(**param_)
                        # print(model_stack[model][0])
                        # print("Training Ending")
                        # corss-validation reslut fit
                        scores = cross_validate(model_stack[model][0], patient_value, np.array(label_), scoring=scoring, cv = 5,  n_jobs = -1, return_train_score=True)
                        scores_stack.append(scores)
                        result_dict['Reg:mae'].append("{} - {}".format(round(np.min(scores['test_mae']),5), round(np.max(scores['test_mae']),5)))
                        result_dict['Reg:mse'].append("{} - {}".format(round(np.min(scores['test_mse']),5), round(np.max(scores['test_mse']),5)))
                        result_dict['Reg:mape'].append("{} - {}".format(round(np.min(scores['test_mape']),5), round(np.max(scores['test_mape']),5))) 
                        
                        # print("pass",model)
                        grid_stack.append(GridModel)
                    # print('----------------------------------------------------')

(84, 80) (84, 80)
Overall:  (84, 160)
Feature Number: 160
ALFF Outcome: AD8主觀認知障礙[0-1,>=2][mix] - cut rank point: 80 - GridSearch: False
(84, 80) (84, 80)
Overall:  (84, 160)
Feature Number: 160
ALFF Outcome: AD8主觀認知障礙[0-1,>=2][mix] - cut rank point: 80 - GridSearch: False
(84, 80) (84, 80)
Overall:  (84, 160)
Feature Number: 160
ALFF Outcome: AD8主觀認知障礙[0-1,>=2][mix] - cut rank point: 80 - GridSearch: False
(84, 80) (84, 80)
Overall:  (84, 160)
Feature Number: 160
ALFF Outcome: MOCA客觀認知測驗分數[<=23,>=24][mix] - cut rank point: 80 - GridSearch: False
(84, 80) (84, 80)
Overall:  (84, 160)
Feature Number: 160
ALFF Outcome: MOCA客觀認知測驗分數[<=23,>=24][mix] - cut rank point: 80 - GridSearch: False
(84, 80) (84, 80)
Overall:  (84, 160)
Feature Number: 160
ALFF Outcome: MOCA客觀認知測驗分數[<=23,>=24][mix] - cut rank point: 80 - GridSearch: False


In [5]:
for i in grid_stack:
    print(i.best_params_)
    print('----------------------------------------------------------------------------------------------------------------------------------------')

In [6]:
result_df = pd.DataFrame(result_dict) 
result_df.style.set_caption("AD8/MOCA - [{} - {} - Grid: {} - CV: {}]".format(data_value_type ,data_type, grid, cross_v))

,outcome,model,Reg:mae,Reg:mse,Reg:mape
0,"AD8主觀認知障礙[0-1,>=2]",MLPR,0.92812 - 2.52201,1.45969 - 10.45825,0.49195 - 0.64133
1,"AD8主觀認知障礙[0-1,>=2]",XBGR,1.06145 - 1.85554,2.28165 - 5.01661,0.57637 - 0.91705
2,"AD8主觀認知障礙[0-1,>=2]",LGBMR,1.09169 - 1.47753,1.88555 - 2.94632,0.55932 - 0.96047
3,"MOCA客觀認知測驗分數[<=23,>=24]",MLPR,3.83773 - 9.10793,25.18449 - 108.17171,0.16449 - 0.37701
4,"MOCA客觀認知測驗分數[<=23,>=24]",XBGR,1.67335 - 3.31294,4.19592 - 14.05025,0.06924 - 0.13961
5,"MOCA客觀認知測驗分數[<=23,>=24]",LGBMR,1.73502 - 2.69839,4.32615 - 10.46568,0.06612 - 0.11684
